In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim # 最適化アルゴリズムを提供するライブラリ
from torchvision import datasets, transforms # PyTorchの画像処理用拡張ライブラリ
from torch.utils.data import DataLoader # Dataset からバッチ単位でデータを
import matplotlib.pyplot as plt # グラフ描画ライブラリ


transform = transforms.Compose([
    transforms.Resize((32, 32)), # 入力画像をすべて32×32ピクセルに統一
    transforms.ToTensor() # PIL画像を [0, 1] の範囲の torch.FloatTensor に変換
])

train_dataset = datasets.ImageFolder("../data/train", transform=transform) # フォルダ構造をもとに自動でラベル付け

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 1度に64枚ずつ学習に使います（ミニバッチ学習）
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 43)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [2]:
import time

# GPUが使えるならGPUに移行（CPUでも動作可）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# エポック数の設定
num_epochs = 10

# 学習ループ開始
for epoch in range(num_epochs):
    model.train()  # モデルを学習モードに
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 勾配の初期化
        optimizer.zero_grad()

        # 順伝播
        outputs = model(images)

        # 損失の計算
        loss = criterion(outputs, labels)

        # 逆伝播（勾配計算）とパラメータ更新
        loss.backward()
        optimizer.step()

        # 統計
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # エポックごとの結果表示
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100
    print(f"[Epoch {epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}%, Time: {time.time() - start_time:.1f}s")

# 学習済みモデルを保存（.pt形式）
torch.save(model.state_dict(), "gtsrb_model.pt")
print("モデルを gtsrb_model.pt に保存しました。")


[Epoch 1/10] Loss: 1.7465, Acc: 48.90%, Time: 59.9s
[Epoch 2/10] Loss: 0.4242, Acc: 87.66%, Time: 30.7s
[Epoch 3/10] Loss: 0.2120, Acc: 94.08%, Time: 29.8s
[Epoch 4/10] Loss: 0.1408, Acc: 96.17%, Time: 31.0s
[Epoch 5/10] Loss: 0.0983, Acc: 97.40%, Time: 29.9s
[Epoch 6/10] Loss: 0.0731, Acc: 98.12%, Time: 30.1s
[Epoch 7/10] Loss: 0.0676, Acc: 98.19%, Time: 27.7s
[Epoch 8/10] Loss: 0.0475, Acc: 98.68%, Time: 30.7s
[Epoch 9/10] Loss: 0.0420, Acc: 98.89%, Time: 30.4s
[Epoch 10/10] Loss: 0.0424, Acc: 98.78%, Time: 30.6s
モデルを gtsrb_model.pt に保存しました。
